In [1]:
from selenium import webdriver
import time
import os
import pandas as pd
import random

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
url4 = "https://www.glassdoor.com/Job/united-states-software-engineer-intern-jobs-SRCH_IL.0,13_IN1_KO14,38.htm"

In [7]:
service = Service(r'C:\Users\jmpat\OneDrive\Desktop\chromedriver.exe')
driver = webdriver.Chrome(service=service)
driver.get(url4)
driver.implicitly_wait(10)

In [15]:
closeButton = True

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    try:
        load_more_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@data-test='load-more']"))
        )
        driver.execute_script("arguments[0].click();", load_more_button)
        print(load_more_button.text)
        time.sleep(random.randint(3, 6))
#         if(closeButton):
#             try:
#                 close_button = WebDriverWait(driver, 5).until(
#                     EC.element_to_be_clickable((By.CLASS_NAME, "CloseButton"))
#                 )
#                 close_button.click()
#                 closeButton = False
#                 time.sleep(random.randint(2, 4))
#             except NoSuchElementException:
#                 pass
#                 time.sleep(random.randint(2, 5))
            
    except (NoSuchElementException, TimeoutException):
        time.sleep(random.randint(2, 5))
        break

In [20]:
jobs = driver.find_elements(By.CSS_SELECTOR, '[data-test="jobListing"]')

In [21]:
companyNames = []
jobTitle = []
jobLocation = []
datePosted = []
jobLink = []

In [22]:
for job in jobs:
    companyNames.append(job.find_element(By.CLASS_NAME, 'EmployerProfile_employerNameContainer__ptolz').text)
    jobTitle.append(job.find_element(By.CSS_SELECTOR, '[data-test="job-title"]').text)
    jobLocation.append(job.find_element(By.CSS_SELECTOR, '[data-test="emp-location"]').text)
    datePosted.append(job.find_element(By.CSS_SELECTOR, '[data-test="job-age"]').text)
    jobLink.append(job.find_element(By.CSS_SELECTOR, '[data-test="job-title"]').get_attribute('href'))

In [31]:
len(jobLink)

663

In [32]:
companyData = pd.DataFrame(companyNames, columns=["Company"])
titleData = pd.DataFrame(jobTitle, columns=["Title"])
locationData = pd.DataFrame(jobLocation, columns=["Location"])
linkData = pd.DataFrame(jobLink, columns=["Links"])
dateData = pd.DataFrame(datePosted, columns=["Date Posted"])

In [33]:
jobData = pd.DataFrame()
jobData = pd.concat([companyData, titleData, locationData, dateData, linkData], axis=1)
numColumns = (jobData.shape[1]*2)-1

for i in range(1, numColumns, 2):
    jobData.insert(i, f'Empty_{i}', '')
    
jobData.index = jobData.index+1

In [119]:
jobData.to_csv("Glassdoor_SWE_Intern_Jobs.csv")

In [34]:
driver.quit()